<a href="https://colab.research.google.com/github/KOdin2/machine_learning/blob/main/KNN_assessment_with_comments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**KNN**

This code up is used to classify a dataset whose labels are a binary (either 1 and 0) a range of hyperparameters are tried. The best hyperparameters are used to calculate the Gini importance of each feature. The Gini importance allows us to remove features with a rank of 0, which means the model does not find it useful.

**Note:** A uploaded dataset must have one column labeled "Label" with the row values as a 1 or 0 otherwise this code will not work

In [1]:
""" Import libraries """
import matplotlib.pyplot as plt
import numpy as np
import io
import pandas as pd
import sklearn
import csv


""" Import sklearn specific functions """
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

""" Import excel writer to allow excel notebooks to be written """
!pip install xlsxwriter
import xlsxwriter

     |████████████████████████████████| 149 kB 4.9 MB/s 


The following code is used to upload as many files as you want to be classified

In [2]:
from google.colab import files
uploaded = files.upload()

Saving Aluminium_3-8Zor_dataset_updated.csv to Aluminium_3-8Zor_dataset_updated.csv
Saving Aluminium_8-25WF_dataset_updated.csv to Aluminium_8-25WF_dataset_updated.csv
Saving Aluminium_25-75WF_dataset_updated.csv to Aluminium_25-75WF_dataset_updated.csv
Saving Aluminium_FM_dataset_updated.csv to Aluminium_FM_dataset_updated.csv
Saving Brass_3-8Zor_dataset_updated.csv to Brass_3-8Zor_dataset_updated.csv
Saving Brass_BIM_dataset_updated.csv to Brass_BIM_dataset_updated.csv
Saving SSteel_8-25Zur_datset_updated.csv to SSteel_8-25Zur_datset_updated.csv
Saving SSteel_25-75WF_dataset_updated.csv to SSteel_25-75WF_dataset_updated.csv
Saving SSteel_25-75WF_manually_reduced.csv to SSteel_25-75WF_manually_reduced.csv
Saving Zinc_8-25WF_dataset_updated.csv to Zinc_8-25WF_dataset_updated.csv


The best measurements to use for classification are:

* precision
* recall
* accuracy
* f1
* roc_auc

Be careful with precision and recall as you can achieve 100% of one and have a really low other. F1 score is the harmoninc mean of precision and recall therefore should be used instread

**calculate_cross_validation_value**

Is used to fond the largest fold value possible for CV 

In [3]:
def calculate_cross_validation_value(y_train):
  """ aqurie the smallest number group of labels and return this for CV """
  arr = y_train.to_numpy()
  bin_arr = np.bincount(arr)
  return bin_arr.min()

**grid_search_RF_function**

This function is used to perfrom a grid search of defined hyperparameters

Once complete this model will the models best parameters

In [13]:
def grid_search_function(x_train, y_train, Varible_list_1 , number_of_cv_folds):

  """ Initalise the KNN model with the best hyperparameters and setup the MinMaxscaler() """
  model = KNeighborsClassifier()
  min_max = MinMaxScaler()
  
  """ Setup the pipeline to perform scaling and model """
  pipe = Pipeline(steps = [
                               ("min_max" , min_max ),
                               
                               ("model" , model )
                               
  ])

  """ Set up the parameters for grid search with desired varibles for it to try """
  param_grid = [
        {'model__n_neighbors': Varible_list_1, }
  ]

  """ Assign the scoring type for grid search """
  scoring_type = 'f1'
  
  """ Set up the gridsearch using defined varible """
  grid_search = GridSearchCV(estimator= pipe, param_grid= param_grid,  cv= number_of_cv_folds , scoring=scoring_type,  return_train_score=True, n_jobs=-1) #n_jobs=-1 will use all avaliable processors avaliable

  """ Perfrom grid search with the x and y train data """
  grid_search.fit(X= x_train, y= y_train)

  """ Aquire the best score and round to 4 decimal places and print the best parameters"""
  best_accuracy = round(grid_search.best_score_,4)
  print("[INFO] Best score: " + str(best_accuracy) + " with parameters: "  + str(grid_search.best_params_) )

  """ Create a list of the best results from the Gridsearch """
  best_model_details = ([ "KNN",
                        scoring_type,
                        best_accuracy,
                        grid_search.best_params_['model__n_neighbors'],
                        str(number_of_cv_folds.n_splits),
                      
  ])

  """ Resturn the best model results and the parameters that will be used for feature reduction """ 
  return best_model_details, grid_search.best_params_

**best_model_test**

THis functions uses the best hyperparameters and the reduced dataset to perform cross validation.

This function then returns the results and are used as the final result

In [5]:
def best_model_test(x_train, y_train, number_of_cv_folds, best_paramater):

  """ Initalise the KNN model with the best hyperparameters and setup the MinMaxscaler() """
  model = KNeighborsClassifier( n_neighbors = best_paramater['model__n_neighbors'])  
  min_max = MinMaxScaler()
  
  """ Setup the pipeline to perform scaling and model """
  pipe = Pipeline(steps = [
                               ("min_max" , min_max ),
                               
                               ("model" , model )
                               
  ])

  """ Use cross_val_predict to perform cross validation and get the predicted labels """
  y_train_pred = cross_val_predict(pipe, x_train, y_train, cv=number_of_cv_folds)

  """ Use the labels to produce a confusion matrix """
  conf = confusion_matrix(y_train, y_train_pred)
  print(conf)

  """ Calcualte the accuracies to .4dp which are then printed"""
  f1        = round(f1_score(y_train, y_train_pred), 4)
  precision = round(precision_score(y_train, y_train_pred), 4)
  recall    = round(recall_score(y_train, y_train_pred), 4)

  print("F1 score\tPurity\t\tRecovery")
  print(str(f1) + "\t\t" + str(precision) + "\t\t" + str(recall) + "\n\n")
    
  """ Write the actual labels and predicted labels to a list. Work out whether the preidction was correct, if so write correct. This is for easy reference """
  predictions_vs_acutal = []
  for index, row_data in enumerate(y_train):  
      if y_train[index] == y_train_pred[index]:
        correct_str = "Correct"
      else:
        correct_str = "Incorrect"
      predictions_vs_acutal.insert(index, [y_train[index], y_train_pred[index], correct_str]   )


  """ Create a list that has the accuracy results of the best model """
  model_check = ([str(f1), str(precision), str(recall), str(conf), str(len(x_train.columns)),  str(number_of_cv_folds.n_splits)])


  return predictions_vs_acutal, model_check

Headers used for the exported CSV file

In [6]:
def write_results_to_excel (file_name, grid_search_list, model_check, predictions_vs_acutal):

  """ Create headers used for the output file """
  header = ["Algorithm", "Scoring type", "Score", "N neighbours", "Stratified KFold splits"]
  model_check_header = ["F1 score", "Purity", "Recovery", "Confusion matrix", "Number of inputs", "Stratified KFold splits"]
  predicion_header = ["Label" , "Prediction", "Result"]

  """Create the workbook """
  workbook = xlsxwriter.Workbook(str(file_name) + '_knn.xlsx')
  worksheet = workbook.add_worksheet("KNN")

  """ Wrtie the Grid search results """
  worksheet.write(0, 0, "Grid search results")
  for row_num, row_data in enumerate(header):
    worksheet.write(1, row_num, row_data) 
  row = 2
  for row_num, row_data in enumerate(grid_search_list):
      for col_num, col_data in enumerate(row_data):
          worksheet.write(row, col_num, str(col_data))
      row+=1
  row+=1

  """ Write the best model results for final CV test with reduced dataset"""
  worksheet.write(row, 0, "Best model testing result")
  row+=1
  for row_num, row_data in enumerate(model_check_header):
    worksheet.write(row, row_num, row_data)
  row +=1
  for row_num, row_data in enumerate(model_check):
    worksheet.write(row, row_num, row_data) 
  row +=2

  """ Write the predicted values againest actual """
  worksheet.write(row, 0, "Prediction results for the best model")
  row +=1 
  for row_num, row_data in enumerate(predicion_header):
    worksheet.write(row, row_num, row_data) 
  row+=1

  for row_num, row_data in enumerate(predictions_vs_acutal):
      for col_num, col_data in enumerate(row_data):
          worksheet.write(row, col_num, str(col_data))
      row+=1

  """ Close the workboot and download"""
  workbook.close()
  files.download(str(file_name) + '_knn.xlsx') 


In [14]:
print("[INFO] CODE START")

""" Loop through all uploaded files"""
for file_name in uploaded:
    grid_search_list = []  
    reduction_mode = False

    """ Convert the uploaded .csv file to a dataframe """
    print("[INFO] File uploaded: " +str(file_name))
    loaded_file = pd.read_csv(io.BytesIO(uploaded[file_name]))

    """ Create the x and y train datasets"""
    x_train  = loaded_file.drop(['Label'], axis='columns')
    y_train     = loaded_file.Label

    """ Calculate the largest of number of fold for StratifiedKfold cross-vaildation"""
    number_of_cv_folds = StratifiedKFold(calculate_cross_validation_value(y_train)) #This function can be changed to a int if a dataset is large as it will take long to run, e.g. 10
    print("[INFO] Number of CV folds: " + str(number_of_cv_folds.n_splits))

    """ Create hyperpameter range half the number of features """
    n_neighbours_var = range(1, int(len(x_train.index)/2))
    print("[INFO] n_neighbours combinations: " + str(n_neighbours_var))

    """ Perform Gird search for the best hyperparameters and appended results to a list"""
    print("[INFO] Finding best model parameters")
    grid_search_result, best_paramater = grid_search_function(x_train, y_train, n_neighbours_var, number_of_cv_folds)   
    grid_search_list.append(grid_search_result)

    """ Using the best parameters and reduced dataset aquire the accuracy results and preductions """
    print("[INFO] Final cross-fold valudation test using the best parameters")
    predictions_vs_acutal, model_check = best_model_test(x_train, y_train, number_of_cv_folds, best_paramater,)
    
    """ Write all the results of the .csv file to an excel work book """
    write_results_to_excel(file_name, grid_search_list, model_check, predictions_vs_acutal)  


[INFO] CODE START
[INFO] File uploaded: Aluminium_3-8Zor_dataset_updated.csv
[INFO] Number of CV folds: 13
[INFO] n_neighbours combinations: range(1, 25)
[INFO] Finding best model parameters
[INFO] Best score: 0.7949 with parameters: {'model__n_neighbors': 1}
[INFO] Final cross-fold valudation test using the best parameters
[[32  5]
 [ 1 12]]
F1 score	Purity		Recovery
0.8		0.7059		0.9231




<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

[INFO] File uploaded: Aluminium_8-25WF_dataset_updated.csv
[INFO] Number of CV folds: 23
[INFO] n_neighbours combinations: range(1, 24)
[INFO] Finding best model parameters
[INFO] Best score: 0.8116 with parameters: {'model__n_neighbors': 15}
[INFO] Final cross-fold valudation test using the best parameters
[[16  7]
 [ 3 23]]
F1 score	Purity		Recovery
0.8214		0.7667		0.8846




<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

[INFO] File uploaded: Zinc_8-25WF_dataset_updated.csv
[INFO] Number of CV folds: 21
[INFO] n_neighbours combinations: range(1, 24)
[INFO] Finding best model parameters
[INFO] Best score: 0.7302 with parameters: {'model__n_neighbors': 9}
[INFO] Final cross-fold valudation test using the best parameters
[[21  7]
 [ 4 17]]
F1 score	Purity		Recovery
0.7556		0.7083		0.8095




<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>